In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime as dt

In [2]:
url = "https://dinkes.papuabaratprov.go.id/"
html_content = requests.get(url).text
soup_url = BeautifulSoup(html_content,"html.parser")
url_data = str(soup_url.find('div',class_='col-xs-12 col-sm-12 col-md-12 col-lg-4').a['href'])
html = requests.get(url_data).text
soup= BeautifulSoup(html,"html.parser")

In [3]:
#Covid
covid = soup.find("div",class_="blog-tiltle_block")
positif_all = []
negatif_all = []
sembuh_all = []
meninggal_all = []
pdp_all = []
odp_all = []
for x in range(0,len(covid.find_all('ol'))):
    for i in range(0,len(covid.find_all('ol')[x].find_all('li'))):
        list = covid.find_all('ol')[x].find_all('li')[i].text.replace('Kab.','Kabupaten').replace('.','').split()
        if x== 0:
            positif_all.append(list)
        elif x== 1:
            negatif_all.append(list)
        elif x==2 :
            sembuh_all.append(list)
        elif x==3 :
            meninggal_all.append(list)
        elif x==4 :
            pdp_all.append(list)
        elif x==5 :
            odp_all.append(list)
all = [positif_all,sembuh_all,meninggal_all,pdp_all,odp_all]
data_all = []
for y in range(0,len(all)):
    for z in all[y]:
        data_all.append(z)
data_kabkot = []
for match in data_all:
    if "Sorong" in match:
        if "Selatan" in match:
            data_kabkot.append(match)
data = []
for i in range(0,len(data_kabkot)):
    for item in data_kabkot[i]:
        if item.isnumeric():
            data.append(item)
col = ["positif","sembuh","meninggal","total_pdp","total_odp"]
df0 = pd.DataFrame(data=[data])
df0['s']=0

In [5]:
#Date
date = soup.find("div",class_="blog-tiltle_block").find_all('p')[-1].text.replace('/'," ").replace(')',"").split()
del date[0]
tanggal = date
tanggal_update = tanggal[2]+'-'+tanggal[1]+'-'+tanggal[0]
date_update = dt.strptime(tanggal_update, '%Y-%m-%d').strftime('%Y-%m-%d')
scrape_date = dt.now().strftime('%Y-%m-%d')
#Other
kabkot = "Sorong Selatan"
provinsi = 'Papua Barat'
types = 'kabkot'
user_pic = 'dimas ananda'

In [6]:
df = {'scrape_date': scrape_date,
      'date_update': date_update,
        'provinsi': provinsi,
        'kabkot' : kabkot,
        'kecamatan': "",
        'kelurahan':"",
        'total_odp' : df0.iloc[0,4],
        'total_pdp': df0[3],
        'total_positif':df0[0],
        'positif_sembuh':df0[1],
        'positif_dirawat':"",
        'positif_isolasi':"",
        'positif_meninggal':df0[2],
        'types':types,
        'source_link': url,
        'user_pic': user_pic}
df = pd.DataFrame(df) 

In [7]:
df

,scrape_date,date_update,provinsi,kabkot,kecamatan,kelurahan,total_odp,total_pdp,total_positif,positif_sembuh,positif_dirawat,positif_isolasi,positif_meninggal,types,source_link,user_pic
0,2021-05-16,2021-05-16,Papua Barat,Sorong Selatan,,,0,0,182,164,,,11,kabkot,https://dinkes.papuabaratprov.go.id/,dimas ananda
